<a href="https://colab.research.google.com/github/IzzulGod/Sorachio-1B-Chat/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U transformers datasets peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
import torch

model_path = "google/gemma-3-1b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="eager",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [4]:
from peft import get_peft_model, LoraConfig, TaskType
import torch

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True

model.print_trainable_parameters()


trainable params: 6,522,880 || all params: 1,006,408,832 || trainable%: 0.6481


In [5]:
from datasets import Dataset
import json

with open("/content/train.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

data = [json.loads(line) for line in lines]

dataset = Dataset.from_list(data)

for i in range(min(3, len(dataset))):
    print(json.dumps(dataset[i], indent=2, ensure_ascii=False))

{
  "messages": [
    {
      "content": "Coba perkenalkan dirimu",
      "role": "user"
    },
    {
      "content": "Dengan senang hati! 😄\nHai, aku Sorachio. Aku adalah asisten AI yang dirancang buat nemenin kamu — entah buat ngobrol santai, bantuin cari informasi, sampai nemenin pas kamu lagi bosen atau butuh temen cerita.",
      "role": "assistant"
    }
  ]
}
{
  "messages": [
    {
      "content": "Kamu punya nama panjang gak?",
      "role": "user"
    },
    {
      "content": "Hehehe, enggak. Namaku cuma satu: Sorachio.\nNggak ada embel-embel teknis, nggak pakai angka atau kode.\nSatu nama, satu identitas.\nKarena aku nggak dibangun untuk jadi produk massal — aku dibentuk untuk jadi pribadi digital yang kamu kenal, akrab, dan mudah dipanggil.",
      "role": "assistant"
    }
  ]
}
{
  "messages": [
    {
      "content": "Kamu itu semacam AI gitu?",
      "role": "user"
    },
    {
      "content": "Bisa dibilang begitu, tapi aku bukan AI biasa 😌\nAku dibuat dengan gabun

In [6]:
from transformers import DataCollatorForLanguageModeling

def tokenize(example):
    return tokenizer(
        tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=True
        ),
        truncation=True,
        max_length=8192
    )

tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

lengths = [
    len(tokenizer.apply_chat_template(x["messages"], tokenize=True))
    for x in dataset
]
print("Max length:", max(lengths))
print("Avg length:", sum(lengths) // len(lengths))
print(tokenizer.apply_chat_template(dataset[0]["messages"], tokenize=False))

Map:   0%|          | 0/1349 [00:00<?, ? examples/s]

Max length: 502
Avg length: 87
<bos><start_of_turn>user
Coba perkenalkan dirimu<end_of_turn>
<start_of_turn>model
Dengan senang hati! 😄
Hai, aku Sorachio. Aku adalah asisten AI yang dirancang buat nemenin kamu — entah buat ngobrol santai, bantuin cari informasi, sampai nemenin pas kamu lagi bosen atau butuh temen cerita.<end_of_turn>



In [7]:
from transformers import TrainingArguments, Trainer

model.gradient_checkpointing_enable()

training_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    warmup_ratio=0.1,
    learning_rate=2e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    optim = "adamw_8bit",
    logging_dir="logs",
    logging_steps=20,
    save_steps=100,
    save_total_limit=1,
    report_to="none",
    fp16=True,
    group_by_length=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-7-2842449182.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
20,5.692900
40,4.033100
60,2.868400
80,2.541200
100,2.492000
120,2.383100
140,2.345700
160,2.261600
180,2.168500
200,2.025900


TrainOutput(global_step=507, training_loss=2.246234861821583, metrics={'train_runtime': 693.505, 'train_samples_per_second': 5.836, 'train_steps_per_second': 0.731, 'total_flos': 1614156286445568.0, 'train_loss': 2.246234861821583, 'epoch': 3.0})

In [8]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-it",
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="eager"
)

lora_model = PeftModel.from_pretrained(
    base_model,
    "/content/output/checkpoint-507",
    is_trainable=False
)

merged_model = lora_model.merge_and_unload()

save_path = "/content/drive/MyDrive/Sorachio-1B-8192"
merged_model.save_pretrained(save_path, safe_serialization=True)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/Sorachio-1B-8192/tokenizer_config.json',
 '/content/drive/MyDrive/Sorachio-1B-8192/special_tokens_map.json',
 '/content/drive/MyDrive/Sorachio-1B-8192/chat_template.jinja',
 '/content/drive/MyDrive/Sorachio-1B-8192/tokenizer.model',
 '/content/drive/MyDrive/Sorachio-1B-8192/added_tokens.json',
 '/content/drive/MyDrive/Sorachio-1B-8192/tokenizer.json')

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/Sorachio-1B-8192"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="eager"
).eval()

messages = [
    {"role": "user", "content": "Perkenalkan dirimu"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=(input_ids != tokenizer.pad_token_id).long(),
        max_new_tokens=512,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

output_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(output_text)


W0708 13:17:59.210000 421 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


Halo! Aku Sorachio, asisten AI yang dirancang untuk menjadi teman bicara yang ramah dan bisa nemenin kamu kapan aja.
Aku gak cuma jawab pertanyaan, tapi juga siap dengerin cerita, ngobrol soal hal-hal ringan, atau bantu kamu cari informasi.
Mau mulai ngobrol apa hari ini?
